# Importing Libraries

In [41]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score

# Importing Datasets

In [42]:
train=pd.read_csv("train_dataset.csv")

In [43]:
test=pd.read_csv("test_dataset.csv")

# Extracting values

In [44]:
train.shape

(534, 11)

In [45]:
train.columns

Index(['Unnamed: 0', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilyCount',
       'Title', 'Has Cabin', 'IsAlone', 'Survived'],
      dtype='object')

In [46]:
train=train.set_index('Unnamed: 0')
train.shape

(534, 10)

In [47]:
X_train=train.iloc[:,0:9]
X_train.head()

,Pclass,Sex,Age,Fare,Embarked,FamilyCount,Title,Has Cabin,IsAlone
Unnamed: 0,,,,,,,,,
711,1,0,29.699118,26.5500,0,0,1,1,0
466,2,0,29.699118,0.0000,0,0,1,0,0
213,2,0,30.000000,13.0000,0,0,1,0,0
496,1,1,54.000000,78.2667,2,1,2,1,1
583,1,0,36.000000,40.1250,2,0,1,1,0


In [48]:
y_train=train.Survived

In [49]:
test=pd.read_csv("test_dataset.csv")

In [50]:
test=test.set_index('Unnamed: 0')

In [51]:
X_test=test.iloc[:,0:9]
y_test=test.Survived

# LogisticRegression Model

In [52]:
lr=LogisticRegression(max_iter=200)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
acc_log =accuracy_score(y_test,y_pred)
acc_log


0.7932960893854749

# Using Cross validation to check performance of the model

In [54]:
#define cross-validation method to use
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [55]:
scores = cross_val_score(lr, X_train, y_train, scoring='neg_mean_absolute_error',cv=cv, n_jobs=-1)

#view mean absolute error
mean(absolute(scores))

0.17568134171907754

In [56]:
#define cross-validation method to use
cv = KFold(n_splits=5, random_state=1, shuffle=True) 

#build logistic regression model
lr = LogisticRegression()

#use LOOCV to evaluate model
scores = cross_val_score(lr, X_train, y_train, scoring='neg_mean_squared_error',cv=cv, n_jobs=-1)

#view RMSE
sqrt(mean(absolute(scores)))

0.4035855870132001

Both MAE and RMSE are quite low meaning our model's performane is quite good

# Pickle the model

In [57]:
# save the model to disc
import pickle

In [58]:
filename = "LogisticRegression model.pkl"
pickle.dump(lr,open(filename,'wb'))